# Bank Telemarketing

## Table of Contents

1. **[Project Background](#1)**,
2. **[Data Cleaning](#2)**,

## Part 1: Project background

Nowadays, marketing expenditure in the banking industry is massive, meaning that it is essential for banks to optimize marketing strategies and improve effectiveness. Understanding customers’ need leads to more effective marketing plans, smarter product designs and greater customer satisfaction.

### Main Objective: increase the effectiveness of the bank's telemarketing campaign

This project will enable the bank to develop a more granular understanding of its customer base, predict customers' response to its telemarketing campaign and establish a target customer profile for future marketing plans.

By analyzing customer features, such as demographics and transaction history, the bank will be able to predict customer saving behaviours and identify which type of customers is more likely to make term deposits. The bank can then focus its marketing efforts on those customers. This will not only allow the bank to secure deposits more effectively but also increase customer satisfaction by reducing undesirable advertisements for certain customers.

## Part 2: Data Cleaning

#### Load the raw data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df1 = pd.read_csv("bank_raw.csv", sep = ';')

There are 45,211 observations in this dataset. Each represents an existing customer that the bank reached via phone calls.

- For each observation, the dataset records 16 input variables that stand for both qualitative and quantitative attributes of the customer, such as age, job, housing and personal loan status, account balance, and the number of contacts.
- There is a single binary output variable that denotes “yes” or “no” revealing the outcomes of the phone calls.

In [2]:
df1.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


#### Cleaning the dataset

#####  Deal with missing data
There is no missing value in this dataset. Nevertheless, there are values like “unknown”, “others”, which are helpless just like missing values. Thus, these ambiguous values are removed from the dataset.

In [4]:
df1.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [6]:
# Step 1: Delete the rows which column 'poutcome' contains 'other'
condition = df1.poutcome == 'other'
df2 = df1.drop(df1[condition].index, axis = 0, inplace = False)

In [7]:
# Step 2: Fill 'unknown' in job and education to 'other'
df2[['job','education']] = df2[['job','education']].replace(['unknown'],'other')

#### 2.2 Drop outliers in the column 'balance'
In order to capture the general trend in the dataset, outliers in the column “balance” are dropped. Outliers are defined as the values which are more than three standard deviations away from the mean. In sum, 2556 rows of data were removed.

In [8]:
from scipy.stats import zscore

df2[['balance']].mean()
df2[['balance']].mean()

df2['balance_outliers'] = df2['balance']
df2['balance_outliers']= zscore(df2['balance_outliers'])

condition1 = (df2['balance_outliers']>3) | (df2['balance_outliers']<-3 )
df3 = df2.drop(df2[condition1].index, axis = 0, inplace = False)

In [9]:
df4 = df3.drop('balance_outliers', axis=1)

#### 2.3 Creating and transforming data

Some changes were made to the column name, units and data types for easier analysis.

In [10]:
# Step 1: Change column name: 'y' to 'response'
df4.rename(index=str, columns={'y': 'response'}, inplace = True)

def convert(df4, new_column, old_column):
    df4[new_column] = df4[old_column].apply(lambda x: 0 if x == 'no' else 1)
    return df4[new_column].value_counts()

convert(df4, "response_binary", "response")

response_binary
0    37785
1     4870
Name: count, dtype: int64

In [11]:
# Step 2: Drop column "contact" which is useless
df5 = df4.drop('contact', axis=1)

In [12]:
# Step 3: Change the unit of 'duration' from seconds to minutes
df5['duration'] = df5['duration'].apply(lambda n:n/60).round(2)

In [13]:
# Step 4: Change 'month' from words to numbers for easier analysis
lst = [df5]
for column in lst:
    column.loc[column["month"] == "jan", "month_int"] = 1
    column.loc[column["month"] == "feb", "month_int"] = 2
    column.loc[column["month"] == "mar", "month_int"] = 3
    column.loc[column["month"] == "apr", "month_int"] = 4
    column.loc[column["month"] == "may", "month_int"] = 5
    column.loc[column["month"] == "jun", "month_int"] = 6
    column.loc[column["month"] == "jul", "month_int"] = 7
    column.loc[column["month"] == "aug", "month_int"] = 8
    column.loc[column["month"] == "sep", "month_int"] = 9
    column.loc[column["month"] == "oct", "month_int"] = 10
    column.loc[column["month"] == "nov", "month_int"] = 11
    column.loc[column["month"] == "dec", "month_int"] = 12

#### 2.4 Filtering

In [14]:
# Step 1: Drop rows that 'duration' < 5s
condition2 = (df5['duration']<5/60)
df6 = df5.drop(df5[condition2].index, axis = 0, inplace = False)

In [16]:
# Step 2: Drop customer values with 'other' education
condition3 = (df6['education'] == 'other')
df7 = df6.drop(df6[condition3].index, axis = 0, inplace = False)

In [19]:
df7.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40841 entries, 0 to 45209
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              40841 non-null  int64  
 1   job              40841 non-null  object 
 2   marital          40841 non-null  object 
 3   education        40841 non-null  object 
 4   default          40841 non-null  object 
 5   balance          40841 non-null  int64  
 6   housing          40841 non-null  object 
 7   loan             40841 non-null  object 
 8   day              40841 non-null  int64  
 9   month            40841 non-null  object 
 10  duration         40841 non-null  float64
 11  campaign         40841 non-null  int64  
 12  pdays            40841 non-null  int64  
 13  previous         40841 non-null  int64  
 14  poutcome         40841 non-null  object 
 15  response         40841 non-null  object 
 16  response_binary  40841 non-null  int64  
 17  month_int        